In [1]:
from dflow import config, s3_config
from dflow.plugins import bohrium
from dflow.plugins.bohrium import TiefblueClient
config["host"] = "https://workflows.deepmodeling.com"
config["k8s_api_server"] = "https://workflows.deepmodeling.com"
bohrium.config["username"] = input("Bohrium username: ")
bohrium.config["password"] = input("Bohrium password: ")
bohrium.config["project_id"] = input("Project ID: ")
s3_config["repo_key"] = "oss-bohrium"
s3_config["storage_client"] = TiefblueClient()

In [1]:
import time, sys
sys.path += ["../.."]
from dflow import download_artifact
from dflow.python import (upload_packages)
from test_mlwf_qe import test_prep_run
upload_packages.append("../../mlwf_op")

In [ ]:
input_setting = {
    'name': 'water',
    'commands': {
        'pw': 'mpirun -n 32 pw.x',
        'pw2wannier': 'mpirun -n 32 pw2wannier90.x',
        'wannier90' : 'mpirun -n 32 wannier90.x', 
        'wannier90_pp' : 'wannier90.x'
    },
    'dft_type': 'qe',
    'dft_params':{
        'cal_type': 'scf',      # scf or scf+nscf
        'qe_params': {
            'control': {
                'title'     : 'water',
            },
            'system': {
                'ecutwfc'   : 110,
                'nosym'     : True,
                'input_dft' : 'PBE',
                'ibrav'     : 0,
            },
            'electrons': {
                'conv_thr'  : 1e-07
            },
            'ions': None,
            'cell': None,
        },
        'pw2wan_params': {
            'inputpp': {
                'spin_component' : None,
                'write_mmn' : True,
                'write_amn' : True,
                'write_unk' : False
            }
        },
        'pseudo_dir': 'pseudo',
        'atomic_species': {
            'O': {
                'mass'  : 16.0,
                'pseudo': 'O_ONCV_PBE-1.2.upf'
            },
            'H': {
                'mass'  : 1.0,
                'pseudo': 'H_ONCV_PBE-1.2.upf'
            },
        },
        'k_grid': (1, 1, 1)
    },
    'wannier90_params': {
        'wan_params': {
            'num_iter'              : 200,
            'num_wann'              : 256,
            'write_xyz'             : True,
            'translate_home_cell'   : True,
            'guiding_centres'       : True,
            "mp_grid"                : "1, 1, 1"
        },
        'projections': {
            'O': 'sp3'
        },
    },
}
machine_setting = {
    'type': 'bohrium',
    'group_size': 2,
    'params': {
        "job_name": "qe-job",
        "backward_files": ["conf*/*.xyz", "conf*/*.wout"],
        "project_id": 11246,
        "image_name": "registry.dp.tech/dptech/prod-13467/wannier-qe:7.0",
        "machine_type": "c32_m64_cpu",
        "job_type": "container",
        "platform": "ali",
        "jgid": 77777
    }
}

In [ ]:
wf = test_prep_run(input_setting, machine_setting)
step = wf.query_step("prepare")[0]
download_artifact(step.outputs.artifacts["task_path"], path="./back")

Workflow has been submitted (ID: prepare-run-test-s4glr, UID: f102b4e9-c6fb-4e3e-bad2-797d19803dbf)


AssertionError: 